# Decision Tree using Entropy and Information Gain
Ryan Miller

### Importing Packages

In [37]:
from scipy import stats
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from time import time

### Reading in and Splitting the Data 

In [2]:
#reading in the Pima Indians Diabetes dataset
data = pd.read_csv('../Data/diabetes.csv')

#splitting data into features and target variables
target = np.array(data.iloc[:,-1]).reshape((-1,1))
features = data.iloc[:,:-1]

#scaling the features to mean 0 and unit variance
ss = StandardScaler()
features = ss.fit_transform(np.array(features))

#adding intercept column to features
features = np.append(features,np.ones((features.shape[0],1)),axis=1)

#splitting the data into train and test sets
X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.2,random_state=0)

### Using Information Gain for Deciding on Decision Tree Split Values
Information Gain = Entropy before Split - Entropy after Split  
Entropy = $ \sum^c_{i=1} -p_i*log_2(p_i) $  
where $c$ is the number of classes and $p_i$ is the probability that an observation belongs to the current class.

In [3]:
#computes entropy for information gain
def entropy(class_y):
    unique_vals = list(np.unique(class_y))
    entropy = 0
    for val in unique_vals:
        prob = sum([1 for i in class_y if i == val])/len(class_y)
        entropy += prob*np.log2(prob)
    return -entropy

#partitioning classes based on given split value and variable
def partition_classes(X, y, split_attribute, split_val):
    X_left = []
    X_right = []
    y_left = []
    y_right = []

    #splitting data when split_val is categorical
    if type(split_val) is str:
        for i in range(len(X)):
            if X[i][split_attribute] == split_val:
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                X_right.append(X[i])
                y_right.append(y[i])
                
    #splitting data when split_val is continuous
    else:
        for i in range(len(X)):
            if X[i][split_attribute] <= split_val:
                X_left.append(X[i])
                y_left.append(y[i])
            else:
                X_right.append(X[i])
                y_right.append(y[i])

    return (X_left, X_right, y_left, y_right)

#computing information gained from the current split
def information_gain(previous_y, current_y):
    #calculating previous entropy
    entropy_prev = entropy(previous_y)

    #calculating current entropy given split
    entropy_cur = 0
    total_len = sum(len(i) for i in current_y)
    for i in current_y:
        entropy_cur += entropy(i)*(len(i)/total_len)

    return entropy_prev - entropy_cur

In [26]:
class DecisionTree(object):
    def __init__(self,depth=0):
        # Initializing the tree as a dictionary with a depth of 0
        self.tree = {'depth':depth}
        pass
    
    #recursively training the decision tree using the given data
    def learn(self, X, y, max_depth=15):
        X_arr = np.array(X)
        #checking stopping conditions (y only has one class or depth is >= given max_depth)
        if len(np.unique(y)) == 1 or self.tree['depth'] >= max_depth:
            self.tree['is_leaf'] = True
            self.tree['class'] = stats.mode(y).mode
        else:
            self.tree['is_leaf'] = False
            #for each attribute, finding split value that gives max information gain
            info_gain_list = []
            split_val_list = []
            
            #looping over all columns of X
            for i in range(X_arr.shape[1]):
                #using mode of categorical values for split
                if type(X_arr[0,i]) is str:
                    split_val = stats.mode(X_arr[:,i]).mode[0]
                #using mean of continuous variables for split
                else:
                    split_val = np.mean(X_arr[:,i])
                #splitting X based on split val and current attribute
                (X_left, X_right, y_left, y_right) = partition_classes(X, y, i, split_val)
                #storing current information gain and split val
                info_gain_list.append(information_gain(y,[y_left,y_right]))
                split_val_list.append(split_val)

            #selecting split attribute that gives max information gain
            split_attribute = np.argmax(info_gain_list)
            split_val = split_val_list[split_attribute]

            #splitting data based on best split attribute
            (X_left, X_right, y_left, y_right) = partition_classes(X, y, split_attribute, split_val)
            
            #storing split attribute and value
            self.tree['split_attribute'] = split_attribute
            self.tree['split_val'] = split_val_list[split_attribute]
            
            #recursively training child nodes
            self.tree['left'] = DecisionTree(depth=self.tree['depth']+1)
            self.tree['left'].learn(X_left,y_left,max_depth)
            self.tree['right'] = DecisionTree(depth=self.tree['depth']+1)
            self.tree['right'].learn(X_right,y_right,max_depth)
            
    #classifying one observation using trained tree
    def classify_one(self, record):
        if self.tree['is_leaf'] == True:
            return self.tree['class'][0]
        else:
            #splitting by categorical split_val
            if type(self.tree['split_val']) is str:
                #deciding split direction
                if record[self.tree['split_attribute']] == self.tree['split_val']:
                    return self.tree['left'].classify_one(record)
                else:
                    return self.tree['right'].classify_one(record)
            #splitting by continuous split_val
            else:
                #deciding split direction
                if record[self.tree['split_attribute']] <= self.tree['split_val']:
                    return self.tree['left'].classify_one(record)
                else:
                    return self.tree['right'].classify_one(record)
                
    #classifying a set of observations
    def classify(self,records):
        return [self.classify_one(i)[0] for i in records] 
    
    #scoring the decision tree based on given test features and labels
    def score(self,X,y):
        y_pred = np.array(self.classify(X)).reshape((-1))
        return np.mean(y_pred==y.reshape((-1)))

### Comparing Performance to Sklearn's DecisionTreeClassifier
The test accuracy of my remade Decision Tree Classifier is comparable to Sklearn's implementation, and unlike Sklearn's version, it is capable of handling categorical data without requiring preprocessing beforehand. The main downside that comes to mind is the noticeably slower runtime.

In [39]:
#sklearn
start = time()
dtc = DecisionTreeClassifier(criterion="entropy",random_state=0,max_depth=25)
dtc.fit(X_train,y_train)
end = time()
print("Sklearn's DecisionTreeClassifier Test Accuracy:",np.round(100*dtc.score(X_test,y_test),2),'%')
print("Sklearn's DecisionTreeClassifier Runtime:",np.round(end-start,6),'seconds')

Sklearn's DecisionTreeClassifier Test Accuracy: 72.08 %
Sklearn's DecisionTreeClassifier Runtime: 0.003964 seconds


In [40]:
#self-made
start = time()
dt = DecisionTree()
dt.learn(X_train,y_train,max_depth=25)
end = time()
print("Self-Made DecisionTree Classifier Test Accuracy:",np.round(100*dt.score(X_test,y_test),2),'%')
print("Self-Made DecisionTree Classifier Runtime:",np.round(end-start,6),'seconds')

Self-Made DecisionTree Classifier Test Accuracy: 75.32 %
Self-Made DecisionTree Classifier Runtime: 0.390928 seconds
